In [3]:
#pip uninstall google-generativeai  # Optional: avoid conflicts if not using old code
#pip install -U google-genai

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^



   ---------------------------------------- 0.0/721.9 kB ? eta -:--:--
   ---------------------------------------- 721.9/721.9 kB 4.9 MB/s  0:00:00

   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ------------- -------------------------- 1/3 [tenacity]
   ------------- -------------------------- 1/3 [tenacity]



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
def create_prompt(question, nodes, instances):
    """Build and format the prompt."""
    formatted_prompt = (
        f"convert the following natural language question into a Cypher query."
        f"Please only output the Cypher.\n"
        f"### Input: {question}\n\n"
        f"### Nodes: {nodes}\n\n"
        f"### Instances: {instances}\n\n"
        f"### Here is the graph schema:\n"
        f"Node properties are the following:\n"
        f"MolecularFunction {{url: STRING, license: STRING, name: STRING, identifier: STRING, source: STRING}},\n"
        f"Pathway {{license: STRING, source: STRING, identifier: STRING, name: STRING, url: STRING}},\n"
        f"Anatomy {{license: STRING, mesh_id: STRING, source: STRING, url: STRING, name: STRING, identifier: STRING, bto_id: STRING}},\n"
        f"PharmacologicClass {{class_type: STRING, license: STRING, source: STRING, url: STRING, identifier: STRING, name: STRING}},\n"
        f"Gene {{license: STRING, chromosome: STRING, url: STRING, source: STRING, description: STRING, identifier: INTEGER, name: STRING}},\n"
        f"Symptom {{license: STRING, name: STRING, source: STRING, identifier: STRING, url: STRING}},\n"
        f"BiologicalProcess {{license: STRING, source: STRING, url: STRING, identifier: STRING, name: STRING}},\n"
        f"Disease {{license: STRING, source: STRING, url: STRING, identifier: STRING, name: STRING}},\n"
        f"Compound {{license: STRING, inchi: STRING, inchikey: STRING, source: STRING, url: STRING, identifier: STRING, name: STRING}},\n"
        f"CellularComponent {{license: STRING, name: STRING, url: STRING, identifier: STRING, source: STRING}},\n"
        f"SideEffect {{license: STRING, source: STRING, url: STRING, name: STRING, identifier: STRING}}\n\n"
        f"### The relationships are the following:\n"
        f"(:Anatomy)-[:EXPRESSES_AeG]->(:Gene),\n"
        f"(:Anatomy)-[:DOWNREGULATES_AdG]->(:Gene),\n"
        f"(:Anatomy)-[:UPREGULATES_AuG]->(:Gene),\n"
        f"(:PharmacologicClass)-[:INCLUDES_PCiC]->(:Compound),\n"
        f"(:Gene)-[:PARTICIPATES_GpMF]->(:MolecularFunction),\n"
        f"(:Gene)-[:PARTICIPATES_GpBP]->(:BiologicalProcess),\n"
        f"(:Gene)-[:COVARIES_GcG]->(:Gene),\n"
        f"(:Gene)-[:REGULATES_GrG]->(:Gene),\n"
        f"(:Gene)-[:INTERACTS_GiG]->(:Gene),\n"
        f"(:Gene)-[:PARTICIPATES_GpPW]->(:Pathway),\n"
        f"(:Gene)-[:PARTICIPATES_GpCC]->(:CellularComponent),\n"
        f"(:Disease)-[:LOCALIZES_DlA]->(:Anatomy),\n"
        f"(:Disease)-[:ASSOCIATES_DaG]->(:Gene),\n"
        f"(:Disease)-[:PRESENTS_DpS]->(:Symptom),\n"
        f"(:Disease)-[:RESEMBLES_DrD]->(:Disease),\n"
        f"(:Disease)-[:DOWNREGULATES_DdG]->(:Gene),\n"
        f"(:Disease)-[:UPREGULATES_DuG]->(:Gene),\n"
        f"(:Compound)-[:UPREGULATES_CuG]->(:Gene),\n"
        f"(:Compound)-[:DOWNREGULATES_CdG]->(:Gene),\n"
        f"(:Compound)-[:BINDS_CbG]->(:Gene),\n"
        f"(:Compound)-[:CAUSES_CcSE]->(:SideEffect),\n"
        f"(:Compound)-[:RESEMBLES_CrC]->(:Compound),\n"
        f"(:Compound)-[:TREATS_CtD]->(:Disease),\n"
        f"(:Compound)-[:PALLIATES_CpD]->(:Disease)\n\n"
        f"### Return only a single Cypher query line. No backticks. No explanation.."
    )
    return formatted_prompt


In [ ]:
import os
import json
import time
from typing import Any, Dict, List
from google import genai  # Use Google GenAI SDK
from google.genai import types

def read_json(file_path: str) -> Any:
    with open(file_path, "r", encoding="utf-8") as fp:
        return json.load(fp)

def write_json_safely(obj, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def prompt_model(message):
    """
    Call Gemini and return text output.
    Note: Gemini SDK handles simple strings or structured content.
    """
    GEMINI_API_KEY = ""

    client = genai.Client(api_key=GEMINI_API_KEY,)
    # Adjusting for Gemini's generate_content structure
    response = client.models.generate_content(
        model="", 
        contents=message,
        config=types.GenerateContentConfig(
        max_output_tokens=5000,
        temperature=0.7,
    ),
)   
    return response.text

def load_checkpoint(path: str) -> Dict[str, Any]:
    if os.path.exists(path):
        return read_json(path)
    return {"done": {}, "errors": []}

def save_checkpoint(path: str, ckpt: Dict[str, Any]):
    write_json_safely(ckpt, path)

def process_file_in_batches(
    input_path: str,
    output_path: str,
    checkpoint_path: str,
    batch_size: int = 10, # Lowered for API stability
    sleep_s: float = 1.0,
    id_key: str = "id",
):
    data: List[Dict[str, Any]] = read_json(input_path)
    ckpt = load_checkpoint(checkpoint_path)

    for idx, inst in enumerate(data):
        if id_key not in inst:
            inst[id_key] = f"{idx}"

    total = len(data)
    processed_now = 0

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        batch = data[start:end]

        for inst in batch:
            inst_id = str(inst[id_key])

            if inst_id in ckpt["done"]:
                inst["Gemini3-Output"] = ckpt["done"][inst_id]
                continue

            try:
                out = prompt_model(create_prompt(inst["rewrite_nl"], inst["nodes"], inst["instance"]))

                inst["Gemini3-Output"] = out
                ckpt["done"][inst_id] = out
                processed_now += 1

            except Exception as e:
                err = {
                    "id": inst_id,
                    "error": repr(e),
                    "rewrite_nl": inst.get("rewrite_nl", None),
                }
                ckpt["errors"].append(err)
                inst["Gemini3-Output"] = None
                inst["Gemini3-Error"] = repr(e)

        save_checkpoint(checkpoint_path, ckpt)
        write_json_safely(data, output_path)

        print(f"[{os.path.basename(input_path)}] Batch {start}-{end}/{total} "
              f"(New: {processed_now}, Total: {len(ckpt['done'])}, Errors: {len(ckpt['errors'])})")

        if sleep_s > 0:
            time.sleep(sleep_s)

# Example Execution
# process_file_in_batches("input.json", "output.json", "ckpt.json")